Example of building a simple CNN for CIFAR10 dataset. 

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import torchvision.models as models

In [20]:
dataset_dir = 'dataset_preprocessed'

In [21]:
# #calculate means and std to be used with normalization
# dataset = ImageFolder(dataset_dir, transform=ToTensor())

# # Compute mean and standard deviation
# means = []
# stds = []
# for img, _ in dataset:
#     means.append(img.mean(dim=(1,2)))
#     stds.append(img.std(dim=(1,2)))

# means = np.stack(means).mean(axis=0)
# stds = np.stack(stds).mean(axis=0)

# print(f"Means: {means}")
# print(f"Stds: {stds}")

means= [0.3337701, 0.35129565, 0.36801142]
stds= [0.16881385, 0.1562263, 0.16852096]


In [22]:
#Load resnet50 and freeze
resnet50 = models.resnet50(pretrained=True)

for param in resnet50.parameters():
    param.requires_grad = False

# Replace the last layer for 2 classes only
features = resnet50.fc.in_features
resnet50.fc = nn.Linear(features, 2)

In [23]:
#hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 0.001


"""
Creates a data transformation pipeline using the transforms.Compose function from the PyTorch library.

"""
transform = transforms.Compose(
    [
     transforms.Resize((256,256)),
     transforms.ToTensor(),
     transforms.Normalize((means[0],means[1],means[2]), (stds[0],stds[1],stds[2]))])

# Load the training and validation datasets in folder 'dataset'
full_data = ImageFolder(dataset_dir, transform=transform)

#Split the dataset
train_size = int(0.85 * len(full_data))
test_size = len(full_data) - train_size

torch.manual_seed(13)
trainDataset, testDataset = torch.utils.data.random_split(full_data, [train_size, test_size])

# Create DataLoader objects to iterate over the datasets in batches
trainloader = torch.utils.data.DataLoader(trainDataset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testDataset, batch_size=32)


In [24]:
net = resnet50
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

# Train the network
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 5 == 4:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5))
            running_loss = 0.0

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_acc = 100.0 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Train Acc: {train_acc:.2f}%')

print('Finished Training')

[1,     5] loss: 0.728
[1,    10] loss: 0.533
[1,    15] loss: 0.526
[1,    20] loss: 0.418
[1,    25] loss: 0.353
Epoch 1/10, Train Acc: 73.18%
[2,     5] loss: 0.314
[2,    10] loss: 0.269
[2,    15] loss: 0.217
[2,    20] loss: 0.256
[2,    25] loss: 0.248
Epoch 2/10, Train Acc: 92.85%
[3,     5] loss: 0.217
[3,    10] loss: 0.240
[3,    15] loss: 0.268
[3,    20] loss: 0.176
[3,    25] loss: 0.190
Epoch 3/10, Train Acc: 94.00%
[4,     5] loss: 0.185
[4,    10] loss: 0.193
[4,    15] loss: 0.143
[4,    20] loss: 0.153
[4,    25] loss: 0.209
Epoch 4/10, Train Acc: 95.40%
[5,     5] loss: 0.208
[5,    10] loss: 0.150
[5,    15] loss: 0.146
[5,    20] loss: 0.186
[5,    25] loss: 0.140
Epoch 5/10, Train Acc: 96.17%
[6,     5] loss: 0.173
[6,    10] loss: 0.181
[6,    15] loss: 0.195
[6,    20] loss: 0.133
[6,    25] loss: 0.125
Epoch 6/10, Train Acc: 94.76%
[7,     5] loss: 0.213
[7,    10] loss: 0.179
[7,    15] loss: 0.361
[7,    20] loss: 0.288
[7,    25] loss: 0.153
Epoch 7/10, Tra

In [25]:
# Test the network on the test data
correct = 0
total = 0
y_test = []
y_pred = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        y_test.extend(labels)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.extend(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [26]:
print('Accuracy of the network on the '+str(test_size)+' test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 139 test images: 96 %


In [27]:

# Save the trained model
PATH = './floodFinal.pth'
torch.save(net.state_dict(), PATH)

In [28]:
from sklearn.metrics import confusion_matrix, classification_report

In [29]:
cm = confusion_matrix(y_test, y_pred)

In [30]:
cm

array([[68,  2],
       [ 3, 66]], dtype=int64)

In [31]:
cr = classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.96      0.97      0.96        70
           1       0.97      0.96      0.96        69

    accuracy                           0.96       139
   macro avg       0.96      0.96      0.96       139
weighted avg       0.96      0.96      0.96       139



In [12]:
#Testing on random images from internet

# model = resnet50
# model.load_state_dict(torch.load('floodFinal.pth'))
# model.eval()


# transform = transforms.Compose(
#     [
#      transforms.Resize((256,256)),
#      transforms.ToTensor(),
#      transforms.Normalize((means[0],means[1],means[2]), (stds[0],stds[1],stds[2]))])

# full_data = ImageFolder('data', transform=transform)

# testloader = torch.utils.data.DataLoader(full_data, batch_size=32)

# correct = 0
# total = 0
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data
#         print(labels)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         print(predicted)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print(correct)
# print(total)

# print('Accuracy of the network on the '+str(test_size)+' test images: %d %%' % (
#     100 * correct / total))

tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
tensor([0, 0, 0, 0, 0, 1, 0, 1, 1, 1])
9
10
Accuracy of the network on the 139 test images: 90 %
